In [1]:
from sklearn.datasets import fetch_20newsgroups
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]
fetch_subset = lambda subset: fetch_20newsgroups(
    subset=subset, categories=categories,
    shuffle=True, random_state=42,
    remove=('headers', 'footers', 'quotes'))
train = fetch_subset('train')
test = fetch_subset('test')

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer()
clf = LogisticRegressionCV()
pipeline = Pipeline([('vec', vec), ('clf', clf)])
pipeline.fit(train['data'], train['target'])

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
   ...2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0))])

In [3]:
from eli5 import explain_weights, explain_prediction
from eli5.formatters import format_as_html, format_as_text, format_html_styles, fields

print(format_as_text(explain_weights(clf, vec, target_names=train['target_names'])))

Explained as: linear model

Features with largest coefficients per class.
Caveats:
1. Be careful with features which are not
   independent - weights don't show their importance.
2. If scale of input features is different then scale of coefficients
   will also be different, making direct comparison between coefficient values
   incorrect.
3. Depending on regularization, rare features sometimes may have high
   coefficients; this doesn't mean they contribute much to the
   classification result for most examples.

y='alt.atheism' top features
----------------------
 +18.161  atheism     
 +16.664  atheists    
 +14.504  religion    
 +14.493  bobby       
 +14.350  matthew     
 +13.392  motto       
 +13.322  atheist     
 +13.071  islam       
 +12.770  nanci       
 +12.251  enviroleague
 +12.059  loans       
 +11.700  satan       
 +11.495  posting     
 +11.199  enlightening
 +11.101  natural     
       …  (6377 more positive features)
       …  (20483 more negative features)
 -

In [4]:
from IPython.core.display import display, HTML
show_html = lambda html: display(HTML(html))
show_html_expl = lambda expl, **kwargs: show_html(format_as_html(expl, include_styles=False, **kwargs))
show_html(format_html_styles())

In [5]:
explain_weights(clf, vec, target_names=train['target_names'])

Explanation(estimator="LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,\n           fit_intercept=True, intercept_scaling=1.0, max_iter=100,\n           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,\n           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)", description="\nFeatures with largest coefficients per class.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='alt.atheism', feature_weights=FeatureWeights(pos=[('atheism', 18.160754703713721), ('atheists', 16.66388154806728), ('religion', 14.503943609653264), ('bobby', 14.492721656251437), ('matthew', 14.350119531813474), ('motto', 13.392103554807203), ('atheist', 13.32177671561683), ('islam', 13.071201003885591), ('nanci', 12.770322624166662), ('enviroleague', 12.25121826288216), ('loans', 12.058506112505277), ('satan', 11.700113917614486), ('posting', 11.494709331433029), ('enlightening', 11.198677524316174), ('natural', 11.100873381892626)], neg=[('space', -18.645398164417685), ('hudson', -12.288020739178371), ('christian', -12.258559468381858), ('order', -11.575240188193241), ('fake', -11.246757997469217)], pos_remaining=6377, neg_remaining=20483), proba=None, score=None, weighted_spans=None), TargetExplanation(target='comp.graphics', feature_weights=FeatureWeights(pos=[('graphics', 26.145807122787552), ('image', 19.047689179891684), ('computer', 17.396477887320621), ('3d', 16.901220021777085), ('file', 16.243188214189409), ('points', 14.122055008234911), ('sgi', 13.23977033681947), ('42', 13.201409481753082), ('hi', 12.431577135129235), ('3do', 11.84318166601139), ('animation', 11.225202535414425), ('using', 11.166667671071565), ('code', 10.93818573804109), ('package', 10.850209657472343), ('screen', 10.711313562572618), ('video', 10.689812769129077), ('68070', 10.657104542708758), ('sphere', 10.604568083999279), ('files', 10.584082246010512)], neg=[('space', -18.266043459048859)], pos_remaining=7904, neg_remaining=18956), proba=None, score=None, weighted_spans=None), TargetExplanation(target='sci.space', feature_weights=FeatureWeights(pos=[('space', 35.804696180267349), ('orbit', 17.691317875742566), ('nasa', 15.215623819665144), ('launch', 15.149102074834172), ('spacecraft', 13.201736671446938), ('mars', 12.879404633376126), ('nick', 12.328778283286853), ('allen', 12.13236681836656), ('moon', 11.880378984582869), ('shuttle', 11.760636096772972), ('dc', 11.661076483225907), ('sci', 10.852855115945106), ('solar', 10.651618957984597)], neg=[('graphics', -17.796533225117827), ('god', -13.411869752499568), ('religion', -12.034599344288027), ('image', -11.837276235847662), ('wrong', -11.165407619969084), ('file', -10.940563573731579), ('3d', -10.696320157201569)], pos_remaining=10087, neg_remaining=16773), proba=None, score=None, weighted_spans=None), TargetExplanation(target='talk.religion.misc', feature_weights=FeatureWeights(pos=[('christian', 19.191630811367187), ('blood', 16.648304143564324), ('fbi', 14.92670924000107), ('christians', 14.199846847891804), ('order', 12.769214562235536), ('hudson', 12.759921102563862), ('christ', 12.368940672454007), ('ekr', 12.102394866909478), ('terrorist', 11.958687068024171), ('koresh', 11.652737647348644), ('dead', 11.533759859198657), ('cult', 11.186234669297235)], neg=[('space', -17.279958527266619), ('atheists', -13.037882680668615), ('it', -12.308497158216829), ('edu', -12.232538791378429), ('thanks', -12.199190566919922), ('get', -11.701215787584351), ('could', -11.573416799131898),

In [6]:
show_html_expl(
    explain_prediction(clf, test['data'][2], vec, target_names=train['target_names']),
    force_weights=True, horizontal_layout=False)

``dense_multitarget=True`` is supported for prediction explanations too, and shows just the top prediction highlighting.

In [7]:
show_html_expl(explain_prediction(clf, test['data'][2], vec, target_names=train['target_names']),
               force_weights=True)

We can hide weights by passing ``force_weights=False`` (they still will be shown if it's impossible to highlight text)

In [8]:
show_html_expl(explain_prediction(clf, test['data'][4], vec, target_names=train['target_names']), force_weights=False)

Show explanations for the winning class for first 10 documents from test data

In [9]:
import numpy as np
for doc in test['data'][:10]:
    expl = explain_prediction(clf, doc, vec, target_names=train['target_names'])
    show_html_expl(expl, force_weights=False)

Weight,Feature
-1.394,<BIAS>
-3.078,Highlighted in text (sum)
Weight,Feature
-1.015,<BIAS>
-3.688,Highlighted in text (sum)
Weight,Feature
+6.059,Highlighted in text (sum)
-1.015,<BIAS>
Weight,Feature
-1.019,<BIAS>


Weight,Feature
-1.394,<BIAS>
-2.570,Highlighted in text (sum)
Weight,Feature
-0.003,Highlighted in text (sum)
-1.015,<BIAS>
Weight,Feature
-0.523,Highlighted in text (sum)
-1.015,<BIAS>
Weight,Feature
-0.438,Highlighted in text (sum)


Now use a vectorizer that skips stopwords

In [10]:
vec_stop = TfidfVectorizer(stop_words='english')
clf_stop = LogisticRegressionCV()
pipeline_stop = Pipeline([('vec', vec_stop), ('clf', clf_stop)])
pipeline_stop.fit(train['data'], train['target'])

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
   ...2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0))])

Words such as "the", "in", "of" are not used as features and are not highlighted

In [11]:
show_html_expl(explain_prediction(clf_stop, test['data'][4], vec_stop, target_names=train['target_names']), force_weights=False)

Weight,Feature
-1.397,<BIAS>
-6.387,Highlighted in text (sum)
Weight,Feature
+6.991,Highlighted in text (sum)
-1.018,<BIAS>
Weight,Feature
-1.016,<BIAS>
-6.670,Highlighted in text (sum)
Weight,Feature
-1.071,<BIAS>
